### <b>The SPKF Steps</b>
$\begin{array}{lccccc}
\hline\rule{0pt}{3.5ex} \text { Method } & \gamma & \alpha_0^{(\mathrm{m})} & \alpha_k^{(\mathrm{m})} & \alpha_0^{(\mathrm{c})} & \alpha_k^{(\mathrm{c})} \\
\text { CDKF } & h & \frac{h^2-L}{h^2} & \frac{1}{2 h^2} & \frac{h^2-L}{h^2} & \frac{1}{2 h^2}\rule[-2ex]{0pt}{5ex}\\
\hline
\end{array}
\\$
$_{\quad {}^*h \text{ may take any positive value. For Gaussian RVs, } h = \sqrt{3}}\\\quad{_{L\text{ is the 
dimension of state space vector }x}}$
### <b>SPKF step 1a:</b> State estimate time update.

- Augmented aposteriori state estimate vector for previous time interval :

$\quad \hat{\chi}{^{a,+}_{k-1}}=\begin{bmatrix}{\hat{\chi}{^+_{k-1}}},&{\bar{w}},&{\bar{v}}\end{bmatrix}$<br><br>
- Augmented aposteriori state covariance estimate vector for previous time interval:

$\quad\sum{^{a,+}_{\tilde{x},k-1}}=\mathrm{diag}({\sum{^{+}_{\tilde{x},k-1}}},\sum{\tilde{w}},\sum{\tilde{v}})$<br><br>
- To generate the p+1 augmented sigma points

$\quad\chi{^{a,+}_{k-1}}=\left\{{\hat{\chi}{^{a,+}_{k-1}}},\quad\hat{\chi}{^{a,+}_{k-1}}+\gamma\sqrt{\sum
{^{a,+}_{\tilde{x},k-1}}},\quad\hat{\chi}{^{a,+}_{k-1}}-\gamma\sqrt{\sum{^{a,+}_{\tilde{x},k-1}}}\right\}\\ \quad \rule[-1.5ex]{0pt}{4.5ex}
\mathcal{X}_{k, i}^{x,-}=f\left(\mathcal{X}_{k-1, i}^{x,+}, u_{k-1}, \mathcal{X}_{k-1, i}^{w,+}\right)
$
$
\begin{aligned}
\quad\hat{x}_k^{-} & =\mathbb{E}\left[f\left(x_{k-1}, u_{k-1}, w_{k-1}\right) \mid \mathbb{Y}_{k-1}\right]
 \approx \sum_{i=0}^p \alpha_i^{(\mathrm{m})} f\left(\mathcal{X}_{k-1, i}^{x,+}, u_{k-1}, \mathcal{X}_
 {k-1, i}^{w,+}\right)
 =\sum_{i=0}^p \alpha_i^{(\mathrm{m})} \mathcal{X}_{k, i}^{x,-}
\end{aligned}
$
### <b>SPKF step 1b:</b> Error covariance time update.

- Using the *apriori* sigma points from step 1a, the *apriori* covariance
estimate is computed as :

$\quad\large{{\Sigma}^{-}_{\tilde{x},k} = \sum_{i=0}^{p} \alpha_i^{(c)} (\mathcal{X}_{k,i}^{x,-} - \hat{x}
_k^{-}) (\mathcal{X}_{k,i}^{x,-} - \hat{x}_k^{-})^T
}$
### <b>SPKF step 1c:</b> Estimate system output $y_k$.
- First, we compute the points :

$\quad
\mathcal{Y}_{k,i} = h(\mathcal{X}_{k,i}^{x,-}, u_k, \mathcal{X}_{k-1,i}^{v,+})
$
- The output estimate is then

$\quad
\hat{y}_k = \mathbb{E}[h(x_k, u_k, v_k) \mid \mathbb{Y}_{k-1}] \approx \sum_{i=0}^{p} \alpha_i^{(m)} h(\mathcal{X}_{k,i}^x, u_k, \mathcal{X}_{k-1,i}^{v,+}) = \sum_{i=0}^{p} \alpha_i^{(m)} \mathcal{Y}_{k,i}.
$
### <b>SPKF step 2a:</b> Estimator gain matrix $L_k$
- To compute the estimator gain matrix, we must first compute the
required covariance matrices.

$\large{\quad
\Sigma_{\tilde{y} , k}=\sum_{i=0}^{p} \alpha_{i}^{( \mathrm{c} )} \big( \mathcal{Y}_{k , i}-\hat{y}_{k} \big) \big( \mathcal{Y}_{k , i}-\hat{y}_{k} \big)^{T}}\\
\rule[-.5ex]{0pt}{3ex}
\large{\quad
\Sigma_{\tilde{x} \tilde{y} , k}^{-}=\sum_{i=0}^{p} \alpha_{i}^{( \mathrm{c} )} \big( \mathcal{X}_{k , i}^{x ,-}-\hat{x}_{k}^{-} \big) \big( \mathcal{Y}_{k , i}-\hat{y}_{k} \big)^{T}}$

- Then, we simply compute $L_k=\sum^-_{\tilde{x}\tilde{y},k}\sum^{-1}_{\tilde{y},k}$

### <b>SPKF step 2b:</b> State estimate measurement update.
- The state estimate is computed as

$\quad\hat{x}^+_k=\hat{x}^-_k+L_k{(y_k-\hat{y}_k)}$

### <b>SPKF step 2c:</b> Error covariance measurement update.
- The final step is calculated directly from the optimal formulation:

$\large\quad\sum{^{+}_{\tilde{x},k}}=\sum_{\tilde{x},k}^--L_k\sum_{\tilde{y},k}L_k^T$

<h3><center>State and Measurement equation for the UKF example ...</center></h3>

$$ x_{k+1} = \sqrt{x_k + 5} + w_k $$
$$ y_k = x_k^3 + v_k $$
$$ \sum\tilde{x}_{\mathrm{true}}=1\sum{\tilde{w}}=0.1 \sum{\tilde{v}}=0.1 $$

In [62]:
import matplotlib.pyplot as plt
import numpy as np
import math
from scipy.linalg import cholesky, inv, sqrtm, block_diag
from scipy import stats
import seaborn as sns

In [63]:
Nx=1 # Number of states
Nxa=3 # Augmented number of states
Ny=1 # Number of measurements
h=np.sqrt(3) # scaling parameter
inv_2h2 = 1 / (2 * h**2) # 1/(2*h^2)
Wmx=np.array([0]) # Wmx0 = (h**2-Nxa) / (h**2) which is essentially 0 for Nxa=3, h=sqrt(3)
Wmx=np.append(Wmx,[inv_2h2]*(2*Nxa)) # Wmxi = 1/(2*h^2) for i=1,...,2*Nxa
rtWmx=np.sqrt(Wmx)

In [64]:
SigmaW=.1 # Process noise standard deviation
SigmaV=.1 # Measurement noise standard deviation
maxIter=4000 # Number of iterations
xtrue=2+np.random.normal(0,1) # Initial true state
xhat=0 # Initial state estimate
# print(xhat)
Sigmax=1 # Initial state covariance estimate

In [65]:
xstore=np.zeros(maxIter+1) # Store true states
xstore[0]=xtrue
xhatstore=np.zeros(maxIter+1) # Store state estimates
xhatstore[0]=xhat
Sigmaxstore=np.zeros(maxIter+1) # Store state covariance estimates
Sigmaxstore[0]=Sigmax
# print(xhat)
# print(np.array([xhat]))

In [66]:
# print(xhat)
for k in range(maxIter):
    # print(k)
    # print(xhat)
    xhata=np.array(xhat)
    # print(xhata)
    xhata=np.append(xhata,[0,0]).reshape(-1,1) # Augmented state vector
    # print(xhata)
    Sigmaxa=block_diag(Sigmax,SigmaW,SigmaV) # Augmented state covariance
    # print(Sigmaxa)
    #cholesky decomposition with lower triangular matrix
    try: #robustness for non-positive definite matrix
        sSigmaxa = cholesky(Sigmaxa, lower=True) 
    except np.linalg.LinAlgError:
        sSigmaxa = np.eye(Sigmaxa.shape[0]) * 1e-6  # fallback
    # print(sSigmaxa)
    #calculate sigma points
    X=np.zeros((Nxa,2*Nxa+1)) # Sigma points matrix
    X[:,0]=xhata[:,0] # First sigma point
    for i in range(Nxa):
        X[:,i+1]=xhata[:,0]+h*sSigmaxa[:,i] # Sigma points for xhat + h*sqrt(Sigmaxa)
        X[:,i+1+Nxa]=xhata[:,0]-h*sSigmaxa[:,i] # Sigma points for xhat - h*sqrt(Sigmaxa)
    Xx=np.sqrt(5+X[0,:])+X[1,:] # Propagate sigma points through state equation
    result=np.sum(Wmx*Xx) # Predicted state mean
    # print(result)
    xhat=result
    Xs=(Xx-xhat)*rtWmx 
    Sigmax=np.sum(Xs**2) # Predicted state covariance
    # print(result1)
    w=np.random.normal(0,math.sqrt(SigmaW)) # Process noise
    v=np.random.normal(0,math.sqrt(SigmaV)) # Measurement noise
    ytrue=xtrue**3+v # True measurement
    xtrue=math.sqrt(5+xtrue)+w # True state
    
    # Y=[Xx[i]**3+X[2,i] for i in range(len(Xx))] # old way
    Y=Xx**3+X[2,:] # Propagate sigma points through measurement equation
    # print(Y)
    result2=np.sum(Wmx*Y)
    yhat=result2 # Predicted measurement mean
    # print(yhat)
    #Ys=np.array([((Y-yhat)*rtWmx) for i in range(len(Y))]) # old way
    Ys=(Y-yhat)*rtWmx
    result2=np.sum(Ys**2)
    SigmaY=result2 # Predicted measurement covariance
    # print(result2)
    # print(len(Xs),len(Ys))
    SigmaXY=np.sum(Xs*Ys) # Cross covariance between state and measurement
    # print(SigmaXY)
    K=SigmaXY/SigmaY # Kalman gain
    # print(K)
    xhat=xhat+K*(ytrue-yhat) # Updated state estimate
    Sigmax=Sigmax-K*SigmaY*K # Updated state covariance estimate
    xstore[k+1]=xtrue # Store true state
    xhatstore[k+1]=xhat # Store state estimate
    Sigmaxstore[k+1]=Sigmax # Store state covariance estimate


In [1]:

xresidual=xstore-xhatstore
resmean=np.mean(xresidual)
resstd=np.std(xresidual)
resskew=stats.skew(xresidual)
reskurt=stats.kurtosis(xresidual)
meanxstore=np.mean(xstore)
meanxhatstore=np.mean(xhatstore)
meanSigmaxstore=np.mean(Sigmaxstore)
print('Residual Mean:', resmean)
print('Residual Std Dev:', resstd)
print('Residual Skewness:', resskew)
print('Residual Kurtosis:', reskurt)
print(f"mean xstore: {meanxstore}")
print(f"mean xhatstore: {meanxhatstore}")
print(f"mean Sigmaxstore: {meanSigmaxstore}")

plt.figure()
sns.histplot(xresidual,kde=True, stat="density",bins=50,color='purple', edgecolor='black', label='Residuals')
plt.axvline(resmean, color='red', linestyle='dashed', linewidth=1, label='Mean')
plt.title('Histogram of Residuals')
plt.xlabel('Residual Value')
plt.ylabel('Density')
plt.legend()

plt.figure(figsize=(10,6))
stats.probplot(xresidual, dist="norm", plot=plt)
plt.title("Q-Q Plot of Residuals")
plt.grid(True)
plt.show()

plt.figure(figsize=(10,6))
params = stats.skewnorm.fit(xresidual)
x = np.linspace(min(xresidual), max(xresidual), 100)
pdf = stats.skewnorm.pdf(x, *params)
plt.plot(x, pdf, label="Fitted Skew-Normal", color='orange')
plt.grid(True)
plt.show()

plt.figure(figsize=(10,6))
plt.plot(xresidual,label='Estimation Error',color='red')
plt.xlabel('Time Step')
plt.legend()
plt.grid()
plt.show()

plt.figure(figsize=(10,6))
plt.plot(xstore,label='True State')
plt.plot(xhatstore,label='Estimated State')
plt.fill_between(np.arange(len(Sigmaxstore)),xhatstore-3*np.sqrt(Sigmaxstore),xhatstore+3*np.sqrt(Sigmaxstore),alpha=0.4,label='Uncertainty')
plt.ylabel('State Value')
plt.title('UKF State Estimation')
plt.legend()
plt.grid()
plt.show()


NameError: name 'xstore' is not defined